<a href="https://colab.research.google.com/github/KhotNoorin/Algorithm-practice/blob/main/RNN_2024KPAD1007.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Noorin Nasir Khot

ID Number: 2024KPAD1007

In [ ]:
# Creating Dataset
english_sentences = [
    "Hello", "How are you?", "What is your name?", "I am learning AI", "Good morning",
    "Good night", "See you later", "Thank you", "You are welcome", "I love programming",
    "This is a book", "That is a pen", "We are friends", "They are playing", "I am hungry",
    "She is reading", "He is writing", "It is raining", "We will go tomorrow", "They came yesterday",
    "I have a dog", "She has a cat", "He likes tea", "We enjoy music", "They eat fruits"
]

hindi_sentences = [
    "<start> नमस्ते <end>", "<start> आप कैसे हैं? <end>", "<start> आपका नाम क्या है? <end>", "<start> मैं AI सीख रहा हूँ <end>", "<start> सुप्रभात <end>",
    "<start> शुभ रात्रि <end>", "<start> बाद में मिलते हैं <end>", "<start> धन्यवाद <end>", "<start> आपका स्वागत है <end>", "<start> मुझे प्रोग्रामिंग पसंद है <end>",
    "<start> यह एक किताब है <end>", "<start> वह एक कलम है <end>", "<start> हम दोस्त हैं <end>", "<start> वे खेल रहे हैं <end>", "<start> मुझे भूख लगी है <end>",
    "<start> वह पढ़ रही है <end>", "<start> वह लिख रहा है <end>", "<start> बारिश हो रही है <end>", "<start> हम कल जाएंगे <end>", "<start> वे कल आए थे <end>",
    "<start> मेरे पास एक कुत्ता है <end>", "<start> उसके पास एक बिल्ली है <end>", "<start> उसे चाय पसंद है <end>", "<start> हमें संगीत पसंद है <end>", "<start> वे फल खाते हैं <end>"
]

In [ ]:
# Preprocess the Data
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [ ]:
# Tokenize the English sentences
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)
eng_vocab_size = len(eng_tokenizer.word_index) + 1

In [ ]:
# Tokenize the Hindi sentences
hin_tokenizer = Tokenizer()
hin_tokenizer.fit_on_texts(hindi_sentences)
hin_sequences = hin_tokenizer.texts_to_sequences(hindi_sentences)
hin_vocab_size = len(hin_tokenizer.word_index) + 1

In [ ]:
# Pad sequences to handle different lengths
max_eng_len = max(len(seq) for seq in eng_sequences)
max_hin_len = max(len(seq) for seq in hin_sequences)

In [ ]:
eng_data = pad_sequences(eng_sequences, maxlen=max_eng_len, padding='post')
hin_data = pad_sequences(hin_sequences, maxlen=max_hin_len, padding='post')

In [ ]:
# Shift Hindi data to create target data
hin_input_data = hin_data[:, :-1]
hin_target_data = hin_data[:, 1:]

In [ ]:
# Build the RNN Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=eng_vocab_size, output_dim=64)(encoder_inputs)
encoder_lstm = LSTM(64, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [ ]:
# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=hin_vocab_size, output_dim=64)(decoder_inputs)
decoder_lstm = LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(hin_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Summary of the model
model.summary()

Model: "functional_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_36            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_37            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_22 (Embedding)  │ (None, None, 64)       │          3,392 │ input_layer_36[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_23 (Embedding)  │ (None, None, 64)       │          3,776 │ input_layer_37[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_22 (LSTM)            │ [(None, 64), (None,    │         33,024 │ embedding_22[0][0]     │
│                           │ 64), (None, 64)]       │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_23 (LSTM)            │ [(None, None, 64),     │         33,024 │ embedding_23[0][0],    │
│                           │ (None, 64), (None,     │                │ lstm_22[0][1],         │
│                           │ 64)]                   │                │ lstm_22[0][2]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, None, 59)       │          3,835 │ lstm_23[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 77,051 (300.98 KB)

 Trainable params: 77,051 (300.98 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Reshape target data to match the model's output shape
hin_target_data = hin_target_data.reshape((hin_target_data.shape[0], hin_target_data.shape[1], 1))

In [ ]:
# Train the model
model.fit([eng_data, hin_input_data], hin_target_data, batch_size=2, epochs=100, validation_split=0.2)

Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 145ms/step - accuracy: 0.1810 - loss: 4.0658 - val_accuracy: 0.1000 - val_loss: 4.0500
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3107 - loss: 3.9942 - val_accuracy: 0.1000 - val_loss: 3.9890
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2800 - loss: 3.7969 - val_accuracy: 0.1000 - val_loss: 3.7113
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2725 - loss: 3.1064 - val_accuracy: 0.1000 - val_loss: 3.4240
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2378 - loss: 2.8259 - val_accuracy: 0.1000 - val_loss: 3.3338
Epoch 6/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2672 - loss: 2.6490 - val_accuracy: 0.1000 - val_loss: 3.2468
Epoch 7/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.3587 - loss: 2.3464 - val_accuracy: 0.1000 - val_loss: 3.2678
Epoch 8/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2410 - loss: 2.5954 - val_accuracy: 0

In [ ]:
# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
# Decoder inference model
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
# Function to translate a sentence
def translate_sentence(input_text):
    input_seq = eng_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_eng_len, padding='post')
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = hin_tokenizer.word_index.get("<start>", 1)

    decoded_sentence = ""
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = hin_tokenizer.index_word.get(sampled_token_index, "")
        if sampled_word == "<end>" or len(decoded_sentence) > 50:
            stop_condition = True
        else:
            decoded_sentence += " " + sampled_word
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

In [ ]:
# Test the translation
test_sentence = "I love programming"
translated_sentence = translate_sentence(test_sentence)
print(f"Translated: {translated_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━